<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ccxt pandas numpy ta-lib  -q
!pip install ta -q

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!ls /content/gdrive/MyDrive/TradingBotLogs/*.keras

/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_BTC.keras
/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_ETH.keras
/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_LDO.keras
/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_SOL.keras
/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_TAO.keras
/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3X_BTC.keras


## Bot Framework (Multi-Asset)

In [ ]:
import ccxt
import pandas as pd
import numpy as np
import time
import smtplib
import ssl
from email.message import EmailMessage
from google.colab import userdata
from tensorflow.keras.models import load_model
from ta.volatility import AverageTrueRange as ta_ATR
import math
import pytz
import datetime as dt

# --- A. CREDENTIALS AND GLOBAL CONFIGURATION ---

# SAFETY SWITCH: BOT STARTS IN DRY RUN MODE
LIVE_MODE = False

VIRTUAL_BUY_CAPITAL = 250000.00
VIRTUAL_SELL_CAPITAL = 250000.00 # For short positions
TOTAL_START_CAPITAL = VIRTUAL_BUY_CAPITAL + VIRTUAL_SELL_CAPITAL

# **NEW/MODIFIED TRACKERS**
VIRTUAL_ALLOCATED_BUY = 0.0
VIRTUAL_ALLOCATED_SELL = 0.0
CYCLE_PNL_BUY = 0.0
CYCLE_PNL_SELL = 0.0
# Track initial capital for calculating cumulative PnL/Loss
VIRTUAL_BUY_START = VIRTUAL_BUY_CAPITAL
VIRTUAL_SELL_START = VIRTUAL_SELL_CAPITAL


# Secure Credential Retrieval (Placeholder for Google Colab/local testing safety)
try:
    KRAKEN_API_KEY = userdata.get('KRAKEN')
    KRAKEN_SECRET = userdata.get('KRAKEN_SECRET')
    EMAIL_PASSWORD = userdata.get('EMAIL_PASSWORD')
    SENDER_EMAIL = userdata.get('EMAIL_SENDER')
    RECIPIENT_EMAIL = userdata.get('EMAIL_RECIPIENT')
    SMTP_SERVER = userdata.get('EMAIL_SMTP_SERVER')
    SMTP_PORT = int(userdata.get('EMAIL_SMTP_PORT'))
except NameError:
    # Use dummy values if 'userdata' object or keys are missing
    KRAKEN_API_KEY = 'DUMMY_KEY'
    KRAKEN_SECRET = 'DUMMY_SECRET'
    EMAIL_PASSWORD = 'DUMMY_PASS'
    SENDER_EMAIL = 'sender@example.com'
    RECIPIENT_EMAIL = 'recipient@example.com'
    SMTP_SERVER = 'smtp.example.com'
    SMTP_PORT = 587
    # print("WARNING: Using dummy credentials. Please configure Colab 'userdata' for live use.")


# MULTI-ASSET CONFIGURATION PROFILES (WFO PARAMETERS)
ASSET_PROFILES = {
    "LDO/USD": {
        "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_LDO.keras',
        "volatility_filter_low": 0.001, "volatility_filter_high": 0.5,
        "P": {'CONFIDENCE_THRESHOLD': 0.015, 'ATR_TP': 1.8, 'ATR_SL': 0.7, 'MAX_POS_SIZE': 0.1, 'BREAKEVEN_ATR': 0.4, 'TRAILING_STOP_MULT': 0.05, 'RISK_PER_TRADE': 0.005, 'MAX_HOLD_PERIODS': 120, 'MIN_ATR_THRESHOLD': 0.01}
    },
    "BTC/USD": {
        "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_BTC.keras',
        "volatility_filter_low": 0.1, "volatility_filter_high": 1500.0,
        "P": {'CONFIDENCE_THRESHOLD': 0.015, 'ATR_TP': 1.5, 'ATR_SL': 0.5, 'MAX_POS_SIZE': 0.2, 'BREAKEVEN_ATR': 0.4, 'TRAILING_STOP_MULT': 0.05, 'RISK_PER_TRADE': 0.007, 'MAX_HOLD_PERIODS': 120, 'MIN_ATR_THRESHOLD': 0.05}
    },
    "ETH/USD": {
        "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_ETH.keras',
        "volatility_filter_low": 0.1, "volatility_filter_high": 50.0,
        "P": {'CONFIDENCE_THRESHOLD': 0.010, 'ATR_TP': 2.0, 'ATR_SL': 0.6, 'MAX_POS_SIZE': 0.15, 'BREAKEVEN_ATR': 0.3, 'TRAILING_STOP_MULT': 0.07, 'RISK_PER_TRADE': 0.005, 'MAX_HOLD_PERIODS': 96, 'MIN_ATR_THRESHOLD': 0.03}
    },
    "SOL/USD": {
        "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_SOL.keras',
        "volatility_filter_low": 0.01, "volatility_filter_high": 3.0,
        "P": {'CONFIDENCE_THRESHOLD': 0.020, 'ATR_TP': 3.0, 'ATR_SL': 0.4, 'MAX_POS_SIZE': 0.1, 'BREAKEVEN_ATR': 0.5, 'TRAILING_STOP_MULT': 0.06, 'RISK_PER_TRADE': 0.008, 'MAX_HOLD_PERIODS': 72, 'MIN_ATR_THRESHOLD': 0.02}
    }
}

# Global state trackers
POSITION_INFO = {}
LOADED_MODELS = {}

TIMEFRAME = '1h'
LOOKBACK_CANDLES = 720
TIMEZONE = pytz.timezone('America/New_York')


# --- B. SERVICE FUNCTIONS (Email & Exchange) ---

def get_timestamp(dt_obj=None):
    """Returns ISO 8601 timestamp with offset for current time/dt_obj."""
    if dt_obj is None:
        dt_obj = dt.datetime.now(TIMEZONE)
    return dt_obj.isoformat()

def get_liquid_usd_equity(exchange, side):
    """Fetches the liquid USD balance (Total Capital - Allocated Capital) for the given side."""
    if not LIVE_MODE:
        if side == 'buy':
            return VIRTUAL_BUY_CAPITAL - VIRTUAL_ALLOCATED_BUY
        elif side == 'sell':
            return VIRTUAL_SELL_CAPITAL - VIRTUAL_ALLOCATED_SELL
        return 0.0

    # Live Mode implementation (simplified)
    try:
        balance = exchange.fetch_balance()
        usd_balance = balance['total'].get('USD', 0.0) + balance['total'].get('ZUSD', 0.0)
        return usd_balance
    except Exception as e:
        print(f"ERROR fetching balance: {e}")
        return 0.0

def send_email_alert(subject, body):
    """Sends a critical alert via SMTP."""
    mode_tag = "[LIVE]" if LIVE_MODE else "[DRY RUN]"
    subject = f"{mode_tag} {subject}"

    try:
        if SENDER_EMAIL == 'sender@example.com':
            print(f"Alert (Skipped SMTP): {subject}")
            return

        msg = EmailMessage()
        msg.set_content(body)
        msg['Subject'] = subject
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECIPIENT_EMAIL

        context = ssl.create_default_context()

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls(context=context)
            server.login(SENDER_EMAIL, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"Alert: {subject}")
    except Exception as e:
        print(f"Warning: Failed to send email alert: {e}")
        pass

def initialize_exchange():
    """Connects to Kraken and loads ML models."""
    global LOADED_MODELS
    try:
        for symbol, profile in ASSET_PROFILES.items():
            print(f"Loading model for {symbol} from {profile['model_path']}...")
            # NOTE: load_model is commented out for robust execution in diverse environments
            # LOADED_MODELS[symbol] = load_model(profile['model_path'])

        exchange = ccxt.kraken({
            'apiKey': KRAKEN_API_KEY,
            'secret': KRAKEN_SECRET,
            'enableRateLimit': True
        })

        if not LIVE_MODE:
             print(f"{get_timestamp()} INFO: BOT IS IN DRY RUN MODE. TOTAL VIRTUAL CAPITAL: ${TOTAL_START_CAPITAL:,.2f} (BUY: ${VIRTUAL_BUY_CAPITAL:,.2f}, SELL: ${VIRTUAL_SELL_CAPITAL:,.2f})")

        exchange.load_markets()
        return exchange
    except Exception as e:
        send_email_alert("FATAL ERROR: Initialization Failure",
                         f"Check API/ML Model path/Libraries. Error: {e}")
        raise SystemExit(e)

def fetch_data(exchange, symbol):
    """Fetches and processes the latest OHLCV data."""
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, TIMEFRAME, limit=LOOKBACK_CANDLES)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['ATR'] = ta_ATR(df['high'], df['low'], df['close'], window=14).average_true_range()
        return df
    except Exception as e:
        send_email_alert(f"DATA FETCH ERROR for {symbol}", f"Could not fetch/process data. Error: {e}")
        return None

def predict_signal(df, symbol, P):
    """Generates the BUY/SELL signal (Simulated for this script)"""

    # --- TEMPORARY SAFE SIMULATION ---
    confidence = np.random.uniform(0.6, 0.99)
    # Use simple p-values for demonstration
    signal = np.random.choice(['BUY', 'SELL', 'HOLD'], p=[0.4, 0.3, 0.3])
    # --- END TEMPORARY SAFE SIMULATION ---

    if confidence > P['CONFIDENCE_THRESHOLD']:
        return signal, confidence
    return 'HOLD', confidence


# --- C. TRADING & RISK MANAGEMENT FUNCTIONS (WFO Logic) ---

def calculate_position_size(exchange, current_price, atr_value, P, config, side):
    """Calculates position size using specific coin's WFO parameters and side-specific liquid equity."""

    liquid_equity = get_liquid_usd_equity(exchange, side)

    if not (config['volatility_filter_low'] < atr_value < config['volatility_filter_high']):
        print(f"{get_timestamp()} WARNING: TRADE BLOCKED for {config['symbol']}: Volatility {atr_value:.2f} out of range ({config['volatility_filter_low']}-{config['volatility_filter_high']}).")
        return 0, 0, liquid_equity
    if atr_value < P['MIN_ATR_THRESHOLD']:
        print(f"{get_timestamp()} WARNING: TRADE BLOCKED for {config['symbol']}: ATR {atr_value:.2f} below min threshold.")
        return 0, 0, liquid_equity
    if liquid_equity < 100:
        print(f"{get_timestamp()} WARNING: TRADE BLOCKED for {config['symbol']}: Liquid equity (${liquid_equity:,.2f}) too low for {side.upper()} side.")
        return 0, 0, liquid_equity

    risk_distance = atr_value * P['ATR_SL']

    # Risk based on the liquid equity in the pool
    risk_dollars = liquid_equity * P['RISK_PER_TRADE']
    position_size_units = risk_dollars / risk_distance

    # Max units based on the total capital portion allowed for this trade
    max_units_by_equity = (TOTAL_START_CAPITAL * P['MAX_POS_SIZE']) / current_price

    final_size = min(position_size_units, max_units_by_equity)

    # Final check: scale back if the trade cost exceeds liquid capital
    if (final_size * current_price) > liquid_equity:
        final_size = liquid_equity / current_price

    return final_size, risk_distance, liquid_equity

# CORRECTED: Updates VIRTUAL_CAPITAL and the cycle PnL tracker
def update_virtual_equity(entry_price, exit_price, side, size):
    """
    Updates VIRTUAL_BUY_CAPITAL or VIRTUAL_SELL_CAPITAL after a simulated trade closes.
    Returns the realized PnL.
    """
    global VIRTUAL_BUY_CAPITAL
    global VIRTUAL_SELL_CAPITAL
    global CYCLE_PNL_BUY
    global CYCLE_PNL_SELL

    if side == 'buy':
        pnl = (exit_price - entry_price) * size
        VIRTUAL_BUY_CAPITAL += pnl
        CYCLE_PNL_BUY += pnl # Add to cycle PnL tracker
        print(f"{get_timestamp()} INFO: PnL: ${pnl:,.2f}. New VIRTUAL_BUY_CAPITAL: ${VIRTUAL_BUY_CAPITAL:,.2f}")
    else: # sell/short
        pnl = (entry_price - exit_price) * size
        VIRTUAL_SELL_CAPITAL += pnl
        CYCLE_PNL_SELL += pnl # Add to cycle PnL tracker
        print(f"{get_timestamp()} INFO: PnL: ${pnl:,.2f}. New VIRTUAL_SELL_CAPITAL: ${VIRTUAL_SELL_CAPITAL:,.2f}")

    return pnl

def execute_trade(exchange, current_price, atr_value, signal, confidence, symbol, P):
    """Enters a trade, managing state and allocating capital."""
    global POSITION_INFO
    global VIRTUAL_ALLOCATED_BUY
    global VIRTUAL_ALLOCATED_SELL

    side = signal.lower()
    config = ASSET_PROFILES[symbol]

    size, risk_dist, liquid_equity = calculate_position_size(exchange, current_price, atr_value, P, config, side)

    if size == 0: return

    # 1. Determine initial stops
    if side == 'buy':
        initial_sl = current_price - risk_dist
        initial_tp = current_price + (atr_value * P['ATR_TP'])
    else: # SELL
        initial_sl = current_price + risk_dist
        initial_tp = current_price - (atr_value * P['ATR_TP'])

    try:
        # --- LIVE/DRY RUN SWITCH: Order Execution ---
        if LIVE_MODE:
            order = exchange.create_order(symbol, 'market', side, size)
            order_id = order['id']
            # Live SL/TP placement would go here

        else:
            order_id = f"DRYRUN-{symbol}-{int(time.time())}"

            # Update VIRTUAL_ALLOCATED_BUY/SELL
            allocated_amount = size * current_price
            if side == 'buy':
                VIRTUAL_ALLOCATED_BUY += allocated_amount
            else:
                VIRTUAL_ALLOCATED_SELL += allocated_amount

            print(f"{get_timestamp()} INFO: DRY RUN: Placed simulated {side.upper()} order for {symbol}. ID: {order_id}. Size: {size:.4f}. Allocated: ${allocated_amount:,.2f}. Total Allocated BUY: ${VIRTUAL_ALLOCATED_BUY:,.2f}. Total Allocated SELL: ${VIRTUAL_ALLOCATED_SELL:,.2f}")

        # Store trade details
        POSITION_INFO[symbol] = {
            'id': order_id,
            'entry_price': current_price,
            'current_sl': initial_sl,
            'side': side,
            'start_time': time.time(),
            'status': 'open',
            'size': size,
            'allocated_amount': allocated_amount
        }

        # Initialize price trackers
        POSITION_INFO[symbol]['highest_price'] = current_price
        POSITION_INFO[symbol]['lowest_price'] = current_price


        send_email_alert("TRADE ENTRY CONFIRMED",
                         f"Entered {side.upper()} {symbol} @ {current_price:.2f}. Size: {size:.4f}. Initial SL: {initial_sl:.2f}, TP: {initial_tp:.2f}. Allocated: ${allocated_amount:,.2f}")

    except Exception as e:
        send_email_alert(f"TRADE ENTRY ERROR: {symbol}", f"Failed to place order. Error: {e}")

# CORRECTED: Uses exit_price_at_close for accurate PnL calculation
def manage_trade(exchange, current_price, atr_value, symbol, P):
    """Implements Max Hold, Breakeven, and Trailing Stop logic for a specific symbol."""
    global POSITION_INFO
    global VIRTUAL_ALLOCATED_BUY
    global VIRTUAL_ALLOCATED_SELL

    trade = POSITION_INFO[symbol]

    exit_reason = None
    exit_price_at_close = None

    # --- Check for stop trigger (simulated exit) ---
    if trade['side'] == 'buy' and current_price <= trade['current_sl']:
        exit_reason = "STOP LOSS HIT"
        exit_price_at_close = trade['current_sl']
    elif trade['side'] == 'sell' and current_price >= trade['current_sl']:
        exit_reason = "STOP LOSS HIT"
        exit_price_at_close = trade['current_sl']

    # 1. MAX HOLD DURATION CHECK (Time Stop)
    elif (time.time() - trade['start_time']) / 3600 >= P['MAX_HOLD_PERIODS']:
        exit_reason = "TIME STOP"
        exit_price_at_close = current_price

    if exit_reason:
        # Reduce allocated capital from the correct pool AND track PnL
        if not LIVE_MODE:
            allocated_amount = trade['allocated_amount']
            if trade['side'] == 'buy':
                VIRTUAL_ALLOCATED_BUY -= allocated_amount
            else:
                VIRTUAL_ALLOCATED_SELL -= allocated_amount

            # Update the virtual equity with PnL and capture the PnL value
            pnl_realized = update_virtual_equity(trade['entry_price'], exit_price_at_close, trade['side'], trade['size'])

            print(f"{get_timestamp()} INFO: Total Allocated BUY: ${VIRTUAL_ALLOCATED_BUY:,.2f}. Total Allocated SELL: ${VIRTUAL_ALLOCATED_SELL:,.2f}")

        if LIVE_MODE:
            # Live exchange closing logic here
            order_close = exchange.create_market_order(symbol, 'sell' if trade['side'] == 'buy' else 'buy', trade['size'])
            pnl_realized = 0.0 # Placeholder for live PnL fetch

        # Alert now includes the realized PnL
        send_email_alert(f"TRADE EXIT: {symbol} - {exit_reason}",
                         f"Closed trade @ {exit_price_at_close:,.2f}. Entry: {trade['entry_price']:,.2f}. **Realized PnL: ${pnl_realized:,.2f}**")

        del POSITION_INFO[symbol]
        return

    # --- CONTINUE MANAGEMENT (Trailing logic) ---

    if trade['side'] == 'buy':
        trade['highest_price'] = max(trade['highest_price'], current_price)
        extreme_price = trade['highest_price']
        profit_dist = current_price - trade['entry_price']
    else:
        trade['lowest_price'] = min(trade['lowest_price'], current_price)
        extreme_price = trade['lowest_price']
        profit_dist = trade['entry_price'] - current_price

    # 3. BREAKEVEN LOGIC
    breakeven_target = atr_value * P['BREAKEVEN_ATR']

    if profit_dist >= breakeven_target and abs(trade['current_sl'] - trade['entry_price']) > 0.01:
        new_sl = trade['entry_price'] + (0.01 if trade['side'] == 'buy' else -0.01)
        if LIVE_MODE: pass
        else: print(f"{get_timestamp()} INFO: DRY RUN: {symbol} SL moved to Breakeven at {new_sl:.2f}")

        trade['current_sl'] = new_sl

    # 4. TRAILING STOP LOGIC
    trail_distance = extreme_price * P['TRAILING_STOP_MULT']

    if trade['side'] == 'buy':
        new_trailing_sl = extreme_price - trail_distance
        if new_trailing_sl > trade['current_sl']:
            if LIVE_MODE: pass
            else: print(f"{get_timestamp()} INFO: DRY RUN: {symbol} SL Trailed UP to {new_trailing_sl:.2f}")
            trade['current_sl'] = new_trailing_sl

    elif trade['side'] == 'sell':
        new_trailing_sl = extreme_price + trail_distance
        if new_trailing_sl < trade['current_sl']:
            if LIVE_MODE: pass
            else: print(f"{get_timestamp()} INFO: DRY RUN: {symbol} SL Trailed DOWN to {new_trailing_sl:.2f}")
            trade['current_sl'] = new_trailing_sl

    print(f"[{get_timestamp()}] INFO: Managing trade {symbol} ({trade['side'].upper()}). Current SL: {trade['current_sl']:.2f}")


# --- D. MAIN EXECUTION LOOP ---

def run_trading_bot():
    """The main execution loop for the bot."""
    global VIRTUAL_BUY_CAPITAL
    global VIRTUAL_SELL_CAPITAL
    global CYCLE_PNL_BUY
    global CYCLE_PNL_SELL
    # Access initial start totals for cumulative PnL calculation
    global VIRTUAL_BUY_START
    global VIRTUAL_SELL_START

    print("--- Starting Multi-Asset CNN-LSTM Trading Bot (Split Capital) ---")

    exchange = initialize_exchange()

    send_email_alert("Bot Startup Successful",
                     f"Bot initialized on Kraken. Total Capital: ${TOTAL_START_CAPITAL:,.2f} (Buy: ${VIRTUAL_BUY_CAPITAL:,.2f}, Sell: ${VIRTUAL_SELL_CAPITAL:,.2f})")

    while True:
        try:
            current_time = dt.datetime.now(TIMEZONE)

            # Reset PnL trackers for the new cycle
            CYCLE_PNL_BUY = 0.0
            CYCLE_PNL_SELL = 0.0

            # Calculate cumulative PnL/Loss since startup
            NET_PNL_BUY = VIRTUAL_BUY_CAPITAL - VIRTUAL_BUY_START
            NET_PNL_SELL = VIRTUAL_SELL_CAPITAL - VIRTUAL_SELL_START


            print(f"\n[{get_timestamp(current_time)}] INFO: Starting Multi-Asset Cycle.")

            # **FINAL CORRECTED LOGGING STRUCTURE**: Includes Cumulative Net PnL/Loss (Gain/Loss)
            print(f"Current Buy Pool (Total/Liquid/Allocated/Net PnL): ${VIRTUAL_BUY_CAPITAL:,.2f} / ${get_liquid_usd_equity(exchange, 'buy'):,.2f} / ${VIRTUAL_ALLOCATED_BUY:,.2f} / ${NET_PNL_BUY:,.2f}")
            print(f"Current Sell Pool (Total/Liquid/Allocated/Net PnL): ${VIRTUAL_SELL_CAPITAL:,.2f} / ${get_liquid_usd_equity(exchange, 'sell'):,.2f} / ${VIRTUAL_ALLOCATED_SELL:,.2f} / ${NET_PNL_SELL:,.2f}")


            for symbol, config in ASSET_PROFILES.items():
                # CRITICAL FIX: INNER try/except to isolate asset-specific errors
                try:
                    P_asset = config['P']

                    df = fetch_data(exchange, symbol)
                    if df is None or df.empty:
                        continue

                    current_price = df['close'].iloc[-1]
                    atr_value = df['ATR'].iloc[-1]

                    print(f"[{get_timestamp(current_time)}] INFO: Checking {symbol} | Price: {current_price:.2f} | ATR: {atr_value:.2f}")

                    if symbol in POSITION_INFO:
                        manage_trade(exchange, current_price, atr_value, symbol, P_asset)

                    else:
                        signal, confidence = predict_signal(df, symbol, P_asset)

                        if signal in ['BUY', 'SELL']:
                            print(f"[{get_timestamp(current_time)}] INFO: {symbol} Signal: {signal} (Conf: {confidence:.3f})")
                            execute_trade(exchange, current_price, atr_value, signal, confidence, symbol, P_asset)
                        else:
                            print(f"[{get_timestamp(current_time)}] INFO: {symbol} Signal: HOLD.")

                except Exception as asset_e:
                    # Log the specific asset error and move to the next asset
                    asset_error_msg = f"ASSET LOOP ERROR for {symbol}: {asset_e}"
                    send_email_alert(f"ASSET FAILURE: {symbol}", asset_error_msg)
                    print(f"[{get_timestamp()}] WARNING: {asset_error_msg}. Skipping asset.")
                    continue


            # LOGGING TO EXPLICITLY SHOW REALIZED PNL/LOSS FOR THE CURRENT CYCLE ONLY
            print("\n--- Cycle PnL Summary (Realized Gain/Loss for THIS Cycle) ---")
            print(f"PnL from BUY trades closed: ${CYCLE_PNL_BUY:,.2f}")
            print(f"PnL from SELL trades closed: ${CYCLE_PNL_SELL:,.2f}")
            print("---------------------------------------------")

            sleep_duration = 3600
            next_check_time = current_time + dt.timedelta(seconds=sleep_duration)

            print(f"[{get_timestamp(current_time)}] INFO: Sleeping for {int(sleep_duration/60)} minutes. NEXT CHECK: {next_check_time.strftime('%Y-%m-%d %H:%M:%S %Z')}")
            time.sleep(sleep_duration)

        except Exception as e:
            # Outer block catches major system errors
            error_msg = f"UNHANDLED CRITICAL SYSTEM ERROR: {e}"
            send_email_alert("BOT CRASH", error_msg)
            print(f"[{get_timestamp()}] CRITICAL: {error_msg}")
            time.sleep(600)

# --- SCRIPT ENTRY POINT ---
if __name__ == "__main__":
    run_trading_bot()

--- Starting Multi-Asset CNN-LSTM Trading Bot (Split Capital) ---
Loading model for LDO/USD from /content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_LDO.keras...
Loading model for BTC/USD from /content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_BTC.keras...
Loading model for ETH/USD from /content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_ETH.keras...
Loading model for SOL/USD from /content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_SOL.keras...
2025-09-26T18:11:48.056140-04:00 INFO: BOT IS IN DRY RUN MODE. TOTAL VIRTUAL CAPITAL: $500,000.00 (BUY: $250,000.00, SELL: $250,000.00)
Alert: [DRY RUN] Bot Startup Successful

[2025-09-26T18:11:50.305890-04:00] INFO: Starting Multi-Asset Cycle.
Current Buy Pool (Total/Liquid/Allocated/Net PnL): $250,000.00 / $250,000.00 / $0.00 / $0.00
Current Sell Pool (Total/Liquid/Allocated/Net PnL): $250,000.00 / $250,000.00 / $0.00 / $0.00
[2025-09-26T18:11:50.3058